# Триангуляция Делоне. Динамический алгоритм: локализация

## Preliminary

Для реализации динамических алгоритмов вставки и удаления вершины из триангуляции нужно уметь по точке быстро находить треугольник из триангуляции, в котором находится эта точка. 

Напомним некоторые базовые определения.

>**Подразбиение Делоне множества точек** — такое разбиение выпуклой оболочки множества точек на множество выпуклых фигур, что в окружности, описанной вокруг любой из фигур, не находится никаких точек из множества.

 

>**Триангуляция Делоне множества точек** — триангуляция, являющаяся подразбиением Делоне.

 

>**Критерий Делоне для ребра** — на ребре можно построить такую окружность, что внутри неё не будет лежать никаких точек.

***Примечание:*** *для понимания дальнейшего материала необходимо сначала ознакомиться с базовой теорией о триангуляции Делоне, однородных координатах, а также со структурой данных DCEL.*(и тут ссылки на ноутбуки)

## Локализационная структура
### Cтруктура
Локализационная структура состоит из нескольких уровней, где каждый уровень — это триангуляция Делоне. На нижнем уровне содержатся все точки. Каждая точка с вероятностью $p$ проходит на следующий уровень (причём если точка — единственная на последнем уровне, то дальше она не пройдёт).

Уровни связаны между собой следующим образом: на уровне $i$ каждая точка содержит указатель на себя же на уровне $i-1$.

<img src="structure_illustration.png" alt="Drawing" style="width: 60%;"/>
На а картинке слева уровень $i - 1$, а справа - уровень $i$.

### Визуализация
Cell ниже запускает визуализацию уровней в локализационной структуре.<br> 
Клик по холсту переносит точку $q$ в место клика. Точка, ближайшая к точке $q$ на этом уровне, будет обозначена синим цветом.<br>
Слайдер задает уровень локализации. 

Если у вас не отображается слайдер, попробуйте запустить jupyter notebook так:
```zsh
pip install ipywidgets
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

Для повторной генерации локализационной структуры просто запустите cell еще раз.

In [ ]:
%matplotlib notebook
from localization_visualizer import visualize
visualize()

### Алгоритм локализации
#### Описание задачи
Нам дают точку $v_{i+1}$, которая на уровне $i+1$ была ближайшей к точке $q$, которую мы локализуем. Нужно получить следующую точку $v_i$, которая будет ближайшей уже на уровне $i$.

Напомним, что триангуляция - это планарный граф, а значит, мы можем хранить ее в памяти так, чтобы за линейное время уметь перебирать все треугольники, инцидентные вершине, а также перебирать все ребра, образующие треугольник.

#### Алгоритм
* Находим, в каком из треугольников, смежных с $v_{i+1}$, лежит отрезок $v_{i+1}$ $q$. Для этого переберем все смежные с $v_{i+1}$ треугольники $ABv_{i+1}$, и для каждого с помощью предиката поворота проверим, лежит ли $v_{i+1}$ $q$ внутри угла $\angle Av_{i+1}B$.
* Последовательно перебирая треугольники, которые пересекает отрезок $v_{i+1}$ $q$, найдем треугольник, в котором лежит $q$.
* Находим ближайшую к $q$ точку. Первым кандидатом на то, чтобы быть ближайшей точкой, становится ближайшая к $q$ вершина найденного в предыдущем пункте треугольника. Для каждого кандидата нужно просмотреть смежные с ним вершины $u_j$. Та из вершин $u_j$, которая окажется ближе всего к $q$, становится следующим кандидатом. Если же среди соседей $u_j$ не нашлось более близких к $q$, значит, $u_j$ и есть искомая ближайшая точка.

<img src="localization1.png" style="float: right;" />
#### Пример
Пусть триангуляция для уровня $i$ выглядит как на рисунке и точка $v_{i+1}$ расположена так же, как на рисунке справа.

На первом шаге алгоритма будет найден треугольник $ABD$.<br>
На втором шаге, последовательно перебирая треугольники, которые пересекает $v_{i+1}$ $q$, мы найдем треугольник $BCD$.<br>
Наконец, на третьем шаге, в качестве кандидата на то, чтобы быть ближайшей точкой, сначала станет точка $C$, а затем - точка $F$.

В итоге алгоритм правильно найдет ближайшую к $q$ точку на уровне $i$. Это точка $F$.

#### Локализация точки, находящейся вне триангуляции
Чтобы отдельно не обрабатывать случай точки, находящейся вне триангуляции, применяют несколько различных подходов.

Один из подходов заключается в том, чтобы "заключить" триангуляцию в другую фигуру, которая гарантированно будет содержать все точки триангуляции. Например, в квадрат. Однако не всегда есть возможность узнать предельные значения координат точек, поэтому более удобным и элегантным является метод, основанный на использовании однородных координат.

Изначально добавим к триангуляции Делоне бесконечную точку $i$, а последующие точки будем рассматривать в однородных координатах. Утверждается, что при таком подходе не нужно отдельно обрабатывать случай локализации точки, находящейся вне триангуляции (разве что, при поиске ближайшей точки на третьем шаге алгоритма, не нужно рассматривать точку $i$, либо считать расстояние до нее равным бесконечности).

Для начала посмотрим как происходит локализация, когда триангуляция состоит ровно из двух точек (не считая $i$), то есть из одного треугольника. Обозначим те его ребра, которые инцидентны $i$, за $a_1$ и $a_2$, а оставшееся ребро - за $b$. Проверку принадлежности точки $q$ треугольнику будем определять по принадлежности точки двум любым углам треугольника, с помощью предиката поворота.

Заметим, что определитель этого предиката для ребер $a_1$ и $a_2$ равен 0 (т.к. одна из строк матрицы нулевая). Это соответствует случаю, когда $i$ лежит на ребре. То есть для углов, инцидентных ребрам $a_1$ и $a_2$, принадлежность точки этим углам вернет $true$. Таким образом, проверка принадлежности точки этому треугольнику полностью определяется тем, по какую сторону от ребра $b$ лежит точка.

<img src="outer_point_case.png" style="float:right">
Это значит, что общем случае, когда триангуляция состоит из более чем двух точек, любая точка, находящяся вне триангуляции, будет лежать внутри какого-нибудь треугольника, инцидентного бесконечной точке. Таких треугольников в общем случае может быть несколько (см. рисунок) и на втором шаге алгоритма может быть выбран не тот, который содержит ближайшую к $q$ точку (например, $ABi$ на рисунке). Однако на третьем шаге все равно будет найден "правильный" треугольник ($BCi$ или $CEi$ на рисунке).

### Корректность алгоритма
<img src="correctness_proof.png" style="float: right;" />
**Теорема** *Данный алгоритм найдёт ближайшую точку.*
<br>$\triangleright$<br>
<div style="padding-left:40px">Предположим, что это не так. Назовём локализуемую точку $q$, а последнего кандидата на то, чтобы быть ближайшей точкой — $v$. Раз эта точка на самом деле не ближайшая, то в окружности $\omega$, проходящей через $v$, с центром в точке $q$ найдутся ещё какие-то точки, не смежные с $v$. Проведём через каждую из них окружность, касающуюся изнутри в точке $v$ окружности $\omega$. Рассмотрим точку $v'$, через которую проходит наименьшая окружность из построенных. В этой окружности не будет лежать никаких точек, так как мы взяли наименьшую. Значит, ребро $vv'$ удовлетворяет критерию Делоне и должно являться ребром триангуляции (по **лемме 1**), но по предположению этого ребра нет. Значит, предположение неверно.</div>
$\triangleleft$

### Реализация
Будем хранить триангуляцию в следующей структуре.

In [ ]:
class vertex:
    def __init__(self, x, y, z, triangles):
        self.x, self.y, self.z = x, y, z # однородные координаты точки
        self.triangles = triangles # список треугольников, инцидентных вершине, в порядке обхода по часовой стрелке

class triangle:
    def __init__(self, vertices, neighbors):
        self.vertices = vertices # лист вершин в порядке обхода по часовой стрелке
        self.neighbors = neighbors # лист смежных по ребру треугольников-соседей, в порядке обхода по часовой стрелке

После выполнения первого шага алгоритма, создадим итератор ***TriangleIterator***, который позволит нам итерироваться по треугольникам триангуляции, которые пересекает луч $qv_{i+1}$. Пусть на каком-то шаге алгоритма (например, на первом) итератор указывает на какой-то треугольник $a$ (на тот треугольник, в котором сейчас "находимся").

Чтобы найти следующий треугольник, переберем всех треугольников-соседей $a$ и найдем из них тот, который не был рассмотрен на предыдущей итерации и любое из ребер которого пересекает $v_{i+1}q$. Стоит заметить, что в случае, если мы используем бесконечную вершину для локализации точки, находящейся вне триангуляции, мы всегда сможем найти такой треугольник - т.е. итератор никогда не остановится.

Отдельно стоит рассмотреть случай, когда $v_{i+1}q$ проходит через вершину. В этом случае нужно запомнить, что мы уже не в треугольнике, а в вершине, и на следующей итерации итератора запускать перебор соседей этой вершины (а не соседей треугольника).

### Отработка практических навыков
Время работы алгоритма локализации - $O(\log n)$; требуемая память - $O(n)$.<br>
Прежде, чем приступить к теоретическому обоснованию данных оценок, читателю предлагается самому реализовать алгоритм.

В коде ниже уже описано несколько вспомогательных функций, заглушка для итератора, а также реализован механизм проверки правильности решения на единственном тесте. Иллюстрация этого теста находится в конце данной секции (сразу после кода).

Читателю предлагается реализовать отдельно каждый из трех шагов алгоритма, заполнив пропуски в телах соответствующих функций.

In [ ]:
from math import sqrt
import next_closest_solution as ncs

'''
Получает на вход a и b типа vertex.
Возвращает расстояние между точками
'''
def dist(a, b):
    # z-координату не учитываем, т.к. у всех точек она равна 1
    return sqrt((a.x - b.x) * (a.x - b.x) + (a.y - b.y) * (a.y - b.y))

'''
Получает на вход a, b, c типа vertex.
Возвращает 1, если тройка (a, b, c) ориентирована по часовой стрелке,
          -1, если тройка ориентирована против часовой стрелки,
           0, если все точки лежат на одной прямой
'''
def orientation(a, b, c):
    return (b.y - a.y) * (c.x - a.x) - (b.x - a.x) * (c.y - a.y)

'''
Возвращает True, если точка d лежит внутри угла abc. Тройка (a, b, c) должна быть ориентирована по часовой стрелке
'''
def in_angle(a, b, c, d):
    return orientation(a, b, d) >= 0 and orientation(c, b, d) <= 0

'''
Возвращает True, если точка d лежит внутри треугольника t. 
'''
def in_triangle(t: triangle, d):
    return in_angle(t.verticies[0], t.verticies[1], t.verticies[2], d) and \
           in_angle(t.verticies[2], t.verticies[0], t.verticies[1], d)
    
'''
Возвращает True, если отрезки ab и cd пересекаются
'''
def intersects(a, b, c, d):
    '''
    Вспомогательная функция. Вовращает true, если на числовой прямой отрезки ab и cd пересекаются
    '''
    def bb (int a, int b, int c, int d) {
        if a > b:
            a, b = b, a
        if c > d:
            c, d = d, c
        return max(a, c) <= min(b, d)
    
    return bb(a.x, b.x, c.x, d.x) and \
           bb(a.y, b.y, c.y, d.y) and \
           orientation(a, b, c) * orientation(a, b, d) <= 0 and \
           orientation(c, d, a) * orientation(c, d, b) <= 0

'''
Итератор, который позволит нам итерироваться по треугольникам, которые пересекает луч qv_{i+1}.
Для более подробной информации см. секцию "Реализация".
'''
class TriangleIterator:
    '''
    initialTriangle - это треугольник, с которого начинается итерация.
    '''
    def __init__(self, initialTriangle, q, vprev):
        self.cur, q, vprev = initialTriangle, q, vprev
    
    def __iter__(self):
        return self

    def next(self):
        prev = self.cur
        
        if isinstance(cur, vertex): # не забываем рассмотреть случай пересечения вершины
            pass
        else:
            pass
        
        return self.prev = prev

'''
эта функция соответствует первому шагу алгоритма
она принимает q и vprev (ближающую к q точку на предыдущем уровне локализации) и 
возвращает итератор, который был описан в секции "Реализация"
q и vprev имеют тип vertex (у q поле triangles равно None)
'''
def find_initial_triangle(q, vprev):
    # insert your code here
    pass
    
'''
эта функция соответствует третьему шагу алгоритма
она принимает q и triangle, в котором находится q
возвращает ближайшую к q точку на текущем уровне локализации
'''
def find_closest_point(q, triangle):
    # insert your code here
    pass

def test_solution(q, vprev):
    initialTriangle = find_initial_triangle(q, vprev)
    iterator = TriangleIterator(initialTriangle, q, vprev)
    for triangle in iterator:
        if in_triangle(triangle, q):
            return find_closest_point(q, triangle)

q, vprev = ncs.test
result = ncs.check_answer(test_solution(q, vprev))
if result == True:
    print("Тест пройден")
else:
    out, ans, comment = result
    print(comment)
    if (type(out) != type(ans)):
        print("Функция должна возвращать значение типа, указанного в описании функции.\n"
              "Вы вернули '{}', а надо '{}'".format(type(out).__name__, type(ans).__name__))
    else:
        objname1 = "фейс" if isinstance(ans, dcel.face) else "точку"
        objname2 = "фейс" if isinstance(ans, dcel.face) else "точка"
        print("Вы вернули {} номер {}, однако правильным ответом является {} номер {}".
              format(objname1, out.name, objname2, ans.name))

Иллюстрация к тесту
<img src="test_illustration.png">

### Время работы, требуемая память
Приведенный выше алгоритм локализации рандомизированный, поэтому доказанные ниже оценки в общем случае будут не верны. Мы будем доказывать их для следующей модели: в триангуляцию была произведена вставка $n$ точек подряд в случайном порядке, каждая точка попала внутрь триангуляции. Для последней точки произведем соответствующие оценки.

**Лемма 1** *Триангуляции Делоне принадлежат те и только те рёбра (с поправкой на точки, лежащие на одной окружности), которые удовлетворяют критерию Делоне.*
<br>$\triangleright$<br><div style="padding-left:40px">
Доказательство этой леммы приведено в коспекте по триангуляции Делоне, с которым читателю рекомендовано было ознакомиться.
</div>$\triangleleft$

#### Память
**Лемма 2** *Матожидание числа уровней в локализационной структуре* — $O(\log n)$
<br>$\triangleright$<br><div style="padding-left:40px">
Для оценки матожидания посчитаем вероятность того, что количество уровней $h$ равно $k$ при вероятности пройти на следующий уровень равной $p$.<br>
$p(h \leq k) = \left(1 - p^{k + 1}\right)^n$, потому что вероятность того, что точка дойдёт до уровня $k + 1$, равна $p^{k + 1}$.<br>
$p(h \geq k) = \left(1 - \left(1 - p^k\right)^n\right)$, потому что вероятность того, что точка не дойдёт до уровня $k$, равна $1 - p^k$.
$p(h = k) = 1 - p(h > k) - p(h < k) = 1 - \left(1 - \left(1 - p^{k + 1}\right)^n\right) - \left(1 - p^{k}\right)^n = \left(1 - p^{k + 1}\right)^n - \left(1 - p^k\right)^n \leq 1 - \left(1 - p^k\right)^n \leq np^k$
$E(h) = \displaystyle\sum_{k = 1}^{\infty} k \cdot p(h = k) = p(1) \cdot 1 + \dots + p\mathopen{}\left(\log_{1/p} n\right)\mathclose{} \cdot \log_{1/p} n + \displaystyle\sum_{k = \log_{1/p} n + 1}^{\infty} k \cdot p(k)$<br>
Оценим первую сумму:<br>
$p(1) \cdot 1 + \dots + p\mathopen{}\left(\log_{1/p} n\right)\mathclose{} \cdot \log_{1/p} n \leq p(1) \cdot \log_{1/p} n + \dots + p\mathopen{}\left(\log_{1/p} n\right)\mathclose{} \cdot \log_{1/p} n = O\mathopen{}\left(\log(n)\right)\mathclose{}$, поскольку сумма этих вероятностей не превосходит единицу.<br>
Оценим вторую сумму:<br>
$\displaystyle\sum_{k = \log_{1/p} n + 1}^{\infty} k \cdot p(k) \leq \displaystyle\sum_{k = \log_{1/p} n}^{\infty} k \cdot n p^k = n \cdot \sum\limits_{k = \log_{1/p} n}^{\infty} k \cdot p^k$<br>
Рассмотрим эту сумму:<br>
$\displaystyle\sum_{k = \log_{1/p} n}^{\infty} k \cdot p^k = p^{\log_{1/p} n} \cdot \displaystyle\sum_{k = 0}^{\infty} \left(k + \log_{1/p} n\right) \cdot p^k = p^{\log_{1/p} n} \cdot $<br>$\left(\sum\limits_{k = 0}^{\infty} \left(k p^k\right) + \log_{1/p} n \cdot \sum\limits_{k = 0}^{\infty} \left(p^k\right)\right) =  p^{\log_{1/p} n} \cdot \left(O(1) + \log_{1/p} n \cdot O(1)\right) = 1/n \cdot O(\log(n))$<br>
Суммируя всё вышесказанное, получаем $O(\log(n))$.
</div>$\triangleleft$

**Теорема** *Локализационная структура занимает $O(n)$ памяти.*<br>
$\triangleright$<br><div style="padding-left:40px">
Триангуляция для $n$ точек занимает $O(n)$ памяти. На нулевом уровне $n$ точек. На уровне $k$ точек $m_k=p \cdot m_{k-1}$. Получим геометрическую прогрессию, сумма которой равна $O(n)$.<br>
</div>$\triangleleft$

#### Время работы
<img src="lemma3.png" style="float: right;" />
**Лемма 3** *Каждая точка на плоскости может являться ближайшей для не более чем шести точек.*
<br>$\triangleright$<br><div style="padding-left:40px">
Предположим, что это не так.

Пусть некоторая точка $u$ является ближайшей для семи точек. Соединим эти семь точек с точкой $u$ отрезками и рассмотрим минимальный из углов, который образуют проведённые отрезки $vu$ и $wu$. Этот угол $\alpha$ меньше $60^{\circ}$ (иначе все семь углов больше либо равны $60^{\circ}$ и их сумма больше $360^{\circ}$).

Так как точка $u$ ближайшая для точек $v$ и $w$, то $vw$ — наибольшая сторона в треугольнике $vwu$. В треугольнике наибольшая сторона лежит напротив наибольшего угла. Но напротив стороны $vw$ лежит угол меньше $60^{\circ}$, значит, сумма углов треугольника меньше $180^{\circ}$. Противоречие. Значит, предположение неверно.
</div>$\triangleleft$

**Лемма 4** *Для заданной точки $q$ на $k$-ом уровне средняя степень ближайшей к ней на $k+1$-ом уровне вершины $u$ равна $O(1)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
*Функция $nn$ принимает точку и множество и возвращает ближайшего соседа заданной точки из заданного множества.*<br>

Рассмотрим некоторый уровень $S_k$. Определим множество $R_k=S_k\cup\{q\}$. Так как мы хотим дать среднюю оценку на степень вершины $u$, то нужно рассмотреть всевозможные варианты множества $S_{k+1}$, для каждого из них посчитать среднюю степень $u$ и результат усреднить. Обозначим всевозможные подмножества  $R_k$, равномощные $R_{k+1}$, за $R'_{k+1}$. Количество таких множеств $={C^{|R_{k+1}|}_{|R_k|}}$. 

Средняя степень вершины $u$ на уровне $S_{k+1}=R'_{k+1}$, равна $\frac {1} {|R_{k+1}|} \displaystyle\sum_{a_i \in R'_{k+1}} \operatorname{deg_{R_k}}\mathopen{}\left(\operatorname{nn}\left(a_i,R'_{k+1}\backslash\{a_i\}\right)\right)\mathclose{}$<br>
Усредняя по всем уровням $R'_{k+1}$, получаем
$E\mathopen{}\left(\operatorname{deg_{S_k}}\left(\operatorname{nn} \left(q, S_{k+1}\right)\right)\right)\mathclose{} = \frac {1} {C^{|R_{k+1}|}_{|R_k|}} \cdot \displaystyle\sum_{R'_{k+1}\subset R_k} \frac {1} {|R_{k+1}|} \displaystyle\sum_{a_i \in R'_{k+1}} \operatorname{deg_{R_k}}\left(\operatorname{nn}\left(a_i,R'_{k+1}\backslash\{a_i\}\right)\right)$<br>

Назовём графом $NN(\{a_i\})$ двудольный граф, в левой и правой долях содержащий точки $\{a_i\}$, рёбра $uv$ которого означают, что точка $v$ является ближайшей для точки $u$ (точка $u$ лежит в левой доли, точка $v$ лежит в правой доли).<br>
Понятно, что $\displaystyle\sum_{a_i \in R'_{k+1}} \operatorname {deg_{R_k}}\mathopen{}\left(\operatorname{nn}\left(a_i, R'_{k+1}\backslash\{a_i\}\right)\right)\mathclose{} = \displaystyle\sum_{a_i\in R'_{k+1}} \operatorname{deg_{R_k}}(a_i) \cdot \operatorname{deg_{NN\left(R'_{k+1}\right)}}(a_i)$, так как степень каждой вершины $a_i$ учтётся ровно столько раз, сколько рёбер ей инцидентно в правой доли графа $NN$.<br>
$E\mathopen{}\left(\operatorname{deg_{S_k}}\mathopen{}\left(\operatorname{nn} \left(q, S_{k+1}\right)\right)\mathclose{}\right)\mathclose{} = \frac {1} {C^{|R_{k+1}|}_{|R_k|}} \displaystyle\sum_{R'_{k+1}\subset R_k} \frac {1} {|R_{k+1}|} \displaystyle\sum_{a_i\in R'_{k+1}} \operatorname{deg_{R_k}}(a_i) \operatorname{deg_{NN\left(R'_{k+1}\right)}}(a_i)$<br>
По **лемме 3** степень вершины из правой доли графа NN не может быть больше шести.<br>
$E\mathopen{}\left(\operatorname{deg_{S_k}}\mathopen{}\left(\operatorname{nn} \left(q, S_{k+1}\right)\right)\mathclose{}\right)\mathclose{} \le \frac {1} {C^{|R_{k+1}|}_{|R_k|}} \displaystyle\sum_{R'_{k+1}\subset R_k} \frac {1} {|R_{k+1}|} \displaystyle\sum_{a_i\in R'_{k+1}} \operatorname{deg_{R_k}}(a_i) \cdot 6 = \frac {6} {C^{|R_{k+1}|}_{|R_k|} \cdot |R_{k+1}|} \displaystyle\sum_{R'_{k+1}\subset R_k} \displaystyle\sum_{a_i\in R'_{k+1}} \operatorname{deg_{R_k}}(a_i) = 6 \cdot \frac {\sum_{a_i\in R_k} \operatorname{deg}(a_i)} {|R_k|} = O(1)$
</div>$\triangleleft$

**Лемма 5** *Среднее число точек, лежащих в окружности с центром в точке $q$ и проходящей через $v_{k+1}$, равна $O\left(1\right)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
<img src="lemma5.png" style="float: right;" />
Рассмотрим точки триангуляции, содержащиеся на уровнях $k$ и $k+1$. Обозначим их за $a_i$. Точки триангуляции, содержащиеся на уровне $k+1$, условно будем называть *черными*, а точки, содержащиеся только на уровне $k$ - *красными*. Для каждой красной точки $a_i$ построим окружность с центром в точке $a_i$, проходящую через ближайшую к ней черную точку. Докажем, что каждая красная точка попадёт в $O(1)$ таких окружностей. Обозначим какую-нибудь такую красную точку за $w$.

Разделим плоскость на шесть равных частей прямыми, проходящими через точку $w$, и рассмотрим одну из частей. Отсортируем все точки, попавшие в неё, по увеличению расстояния до $w$. Получим такую последовательность точек $\{b_0, b_1, ...\}$, что $|wb_i|\le|wb_{i+1}|$. 

Заметим, что если какая-нибудь точка $b_i$ является черной, то все точки, начиная с $b_{i+1}$ уже не содержат в своей окружности точку $w$. Таким образом, среднее число точек $t$, в окружности которых содержится точка $w$, не больше матожидания количества красных точек до первого появления черной точки, которое равно $O(1)$:<br>
$E(t)\le6\cdot\displaystyle\sum_i \left(1-p\right)^i p = O(1)$<br>

Таким образом, каждая точка содержится в $O(1)$ окружностей, значит, каждая окружность содержит $O(1)$ точек.
</div><br>$\triangleleft$

**Лемма 6** *Среднее число рёбер, пересечённое отрезком $qv_{i+1}$ во втором этапе алгоритма локализации, равно $O\left(1\right)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
Рассмотрим рёбра, пересекающие $qv_{i+1}$, для которых хотя бы одна из граничных точек окажется в окружности с центром в точке $q$, проходящей через $v_{i+1}$. Число таких рёбер не превосходит суммы степеней вершин, лежащих внутри окружности. А по **лемме 5** число таких точек равно $O(1)$. При этом средняя степень вершины равна $O(1)$. Таким образом, число таких рёбер равно $O(1)$.

Докажем, что число рёбер, пересекающих $qv_{i+1}$, для которых обе граничные точки лежат вне окружности, тоже равно $O(1)$. При вставке точки $q$ в триангуляцию для этих рёбер перестанет выполняться критерий Делоне: в любой окружности, построенной на ребре как на хорде, будет содержаться либо точка $q$, либо точка $v_{i+1}$. Поэтому эти рёбра придётся флипнуть. Число флипов при вставке точки равно $O(1)$, поэтому число таких рёбер равно $O(1)$.

Итого число рёбер, пересекающих $qv_{i+1}$, равно $O(1)$.
</div>$\triangleleft$

**Лемма 7** *Среднее число треугольников, посещённых на третьем этапе алгоритма локализации, равно $O(1)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
Каждый рассмотренный треугольник имеет хотя бы одну вершину внутри окружности, проведённой через $v_{i+1}$, с центром в точке $q$. То есть число таких треугольников не больше числа точек внутри этой окружности. Таких точек по **лемме 4** $O(1)$, значит, число треугольников тоже равно $O(1)$.
</div>$\triangleleft$

**Лемма 8** *Локализация точки на каждом уровне происходит за $O(1)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
Докажем, что каждый этап локализации происходит за $O(1)$.

**1 этап**: по **лемме 4** средняя степень вершины $v_{i+1}$ равна $O(1)$, поэтому треугольников, в которых может лежать отрезок $qv_{i+1}$ тоже $O(1)$. Просмотрев их все, за $O(1)$ можно понять, в каком из них лежит отрезок $qv_{i+1}$.

**2 этап**: число рёбер, пересечённых отрезком $qv_{i+1}$, равно $O(1)$ (по **лемме 6**). Поэтому этот этап локализации тоже происходит за $O(1)$.

**3 этап**: число треугольников, посещённых на третьем этапе локализации, равно O(1) (по **лемме 7**).

</div>$\triangleleft$

**Теорема** *Локализация точки в триангуляции происходит за $O(\log n)$.*
<br>$\triangleright$<br><div style="padding-left:40px">
Очевидное следствие из **леммы 2** и **леммы 8**.
</div>$\triangleleft$